In [1]:
import os

In [2]:
%pwd

'c:\\Users\\alvin\\Documents\\Project Data Science\\End-To-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\alvin\\Documents\\Project Data Science\\End-To-End-ML-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/alvin0727/End-To-End-ML-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "alvin0727"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "b9d019bd3526bf93e3920cc9bb1c502db1e8ff34"

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path 
    all_params: dict
    metric_file_name: Path
    target_col: str
    mlflow_uri: str
    

In [24]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)    
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_col=schema.name,
            mlflow_uri="https://dagshub.com/alvin0727/End-To-End-ML-Project.mlflow"
            
        ) 
        return model_evaluation_config

In [26]:
import os 
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlProject.utils.common import save_json

In [27]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        r2 = r2_score(actual, predicted)
        mae = mean_absolute_error(actual, predicted)
        return rmse, r2, mae
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        X_test = test_data.drop(self.config.target_col, axis=1)
        y_test = test_data[self.config.target_col]
         
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(X_test)
            (rmse, r2, mae) = self.eval_metrics(y_test, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "r2": r2, "mae": mae} 
            save_json(path=Path(self.config.metric_file_name), data=scores)  
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                
                # Register the model in the MLflow Model Registry
                # Please refer to the doc for more details:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")       
        

In [28]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-26 02:39:48,533]: INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-04-26 02:39:48,536]: INFO:common:yaml file: params.yaml loaded successfully]
[2025-04-26 02:39:48,548]: INFO:common:yaml file: schema.yaml loaded successfully]
[2025-04-26 02:39:48,549]: INFO:common:created directory at: artifacts]
[2025-04-26 02:39:48,551]: INFO:common:created directory at: artifacts/model_evaluation]
[2025-04-26 02:39:48,963]: INFO:common:json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2025/04/26 02:40:08 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.
